In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow import keras

model = tf.keras.models.load_model("/content/drive/MyDrive/Project_AI/final_cnn.h5")
classes = { 0: 'akie, Actinic keratoses and intraepithelial carcinomae',
            1:'bcc, basal cell carcinoma',
            2:'bkl, benign keratosis-like lesions',
            3: 'df, dermatofibroma',
            4: 'nv, melanocytic nevi',
            5: 'vasc, vascular lesions',
            6: 'mel, melanoma'}

In [6]:
from collections import deque
import argparse
import pickle
import cv2
import numpy as np

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)
# Read Video
vs = cv2.VideoCapture('/content/drive/MyDrive/Project_AI/Akie.mp4') #Directory of video
writer = None
(W, H) = (None, None)
# Loop over frames from the video file stream
while True:
	# Read the next frame from the file
	(grabbed, frame) = vs.read()
	# If the frame was not grabbed, then we have reached the end of the stream
	if not grabbed:
		break
	# If the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]
  # Clone the output frame, then convert it from BGR to RGB ordering, resize the
  # frame to a fixed 28x28, and then perform mean subtraction
	output = frame.copy()
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (28, 28)).astype("float32")
	frame -= mean
  # Make predictions on the frame and then update the predictions queue
	preds = model.predict(np.expand_dims(frame, axis=0))[0] #Model start predict
	Q.append(preds)
	# Perform prediction averaging over the current history of previous predictions
	results = np.array(Q).mean(axis=0)
	i = np.argmax(results)
	label = classes[i]
  # Draw the activity on the output frame
	text = "Prediction is: {}".format(label)
	cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
		2, (0, 0, 255), 5)
	# Check if the video writer is None
	if writer is None:
		# Initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter('/content/drive/MyDrive/Project_AI/Akie_predicted.avi', fourcc, 30, (W, H), True) #Add directory of output video
	# Write the output frame to disk
	writer.write(output)
	key = cv2.waitKey(1) & 0xFF
	# If the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
# release the file pointers
print("Video create successfully!!")
writer.release()
vs.release()

Video create successfully!!
